Скачиваем датасет

!wget https://github.com/ods-ai-ml4sg/proj_news_viz/releases/download/data/gazeta.csv.gz

Или данные можно скачать тут https://github.com/IlyaGusev/gazeta

Пример работы с ним:

from corus import load_ods_gazeta path = 'gazeta.csv.gz' records = load_ods_gazeta(path) next(records)

что надо сделать:

на основе word2vec/fasttext реализовать метод поиска ближайших статей (на вход метода должен приходить запрос (какой-то вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших статей к этому запросу)
Проверить насколько хорошо работают подходы


In [1]:
import warnings 
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings('ignore')

In [2]:
import codecs
fileObj = codecs.open( 'gazeta_test.jsonl', "r", "utf_8_sig" )
text = fileObj.read() # или читайте по строке
fileObj.close()

In [3]:
import pandas as pd

In [4]:
text[:50000].split('"')

['{',
 'url',
 ': ',
 'https://www.gazeta.ru/science/2020/02/14_a_12960289.shtml',
 ', ',
 'text',
 ': ',
 'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел помо

In [5]:
from string import punctuation
punctuation_update = punctuation
punctuation_update = punctuation_update + ': ' + ', '

In [6]:
k = []
for i in text.split('"'):
    if i not in punctuation_update:
        k.append(i)
k[:10]

['url',
 'https://www.gazeta.ru/science/2020/02/14_a_12960289.shtml',
 'text',
 'Американское аэрокосмическое агентство NASA огласило названия четырех космических миссий, которые в скором времени могут быть выбраны для реализации и запуск которых может состояться уже в конце этого десятилетия. Эти четыре проекта стали полуфиналистами конкурса, объявленного среди американских научных команд, в котором участвовало более десяти миссий. Все они были отобраны по критериям потенциальной пользы для науки и технической осуществимости проекта. В рамках программы Discovery NASA занимается планированием миссий, которые призваны дать ответы на фундаментальные вопросы о происхождении тел Солнечной системы и возможному наличию жизни на них. «Эти выбранные миссии могут трансформировать наше восприятие некоторых из наиболее активных и сложных миров в Солнечной системе, — заявил Томас Зурбучен, помощник директора NASA по науке. — Исследование каждого из этих небесных тел поможет раскрыть секреты о том,

In [7]:
text_matr = []
title_matr = []
for i in range(len(k)):
    if k[i] == 'text':
        text_matr.append(k[i+1])
    elif k[i] == 'title':       
        title_matr.append(k[i+1])


In [8]:
df = pd.DataFrame({"title": title_matr, "text": text_matr})

In [9]:
df['union'] = df['title'] + '.' + df['text']

In [11]:
df.title[10], df.text[10]

('Дорогое удовольствие: почему снизился спрос на бензин',
 'Спрос на бензин в России в 2019 году упал на 1%. Об этом сообщают «Известия», публикуя результаты исследования группы «Петромаркет» и агентства «Аналитика товарных рынков». Такое снижение спроса стало рекордом за последние годы. По мнению экспертов, тенденция объясняется не высокими ценами на бензин, а снижением реальных доходов населения. Автомобиль стал «довольно дорогим удовольствием», утверждают исследователи. Машина требует затрат на штрафы, техобслуживание и содержание. В результате россияне пересаживаются на такси и общественный транспорт. Некоторые автомобилисты переходят на более дешевый газ. При этом почти 50% россиян считают, что цены на бензин слишком высоки, и из-за этого машиной приходится пользоваться реже. Об этом свидетельствуют данные сентябрьского опроса ВЦИОМ. Среди других причин – дороговизна в обслуживании авто, выплата страховки и налогов (16%), попытки сэкономить средства (15%), пробки на дорогах (14%),

In [12]:
import pandas as pd
import numpy as np

import gensim.downloader as api
from gensim.models import Word2Vec, FastText
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import string
from sklearn.model_selection import train_test_split
import annoy
import re
from tqdm import tqdm
from tqdm import tqdm_notebook

import gensim.models
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten

morfer = MorphAnalyzer()
sw = get_stop_words("ru")
def preprocessor_text(line):
    line = line.lower()
    line = " ".join(i for i in line.split() if i not in punctuation_update).split()
    line = [morfer.parse(i)[0].normal_form for i in line]
    line = [i for i in line if i not in sw and i != "" and i != "—"]
    return line


In [13]:
pattern = r'[^а-яА-Я0-9]'
df['text'] = df['text'].apply(lambda x: re.sub(pattern, ' ', x))

In [14]:
pattern = r'[^а-яА-Я0-9]'
df['title'] = df['title'].apply(lambda x: re.sub(pattern, ' ', x))

In [16]:
df.title[10], df.text[10]

('Дорогое удовольствие  почему снизился спрос на бензин',
 'Спрос на бензин в России в 2019 году упал на 1   Об этом сообщают  Известия   публикуя результаты исследования группы  Петромаркет  и агентства  Аналитика товарных рынков   Такое снижение спроса стало рекордом за последние годы  По мнению экспертов  тенденция объясняется не высокими ценами на бензин  а снижением реальных доходов населения  Автомобиль стал  довольно дорогим удовольствием   утверждают исследователи  Машина требует затрат на штрафы  техобслуживание и содержание  В результате россияне пересаживаются на такси и общественный транспорт  Некоторые автомобилисты переходят на более дешевый газ  При этом почти 50  россиян считают  что цены на бензин слишком высоки  и из за этого машиной приходится пользоваться реже  Об этом свидетельствуют данные сентябрьского опроса ВЦИОМ  Среди других причин   дороговизна в обслуживании авто  выплата страховки и налогов  16    попытки сэкономить средства  15    пробки на дорогах  14   

In [21]:
sentences = []
counter = 0

for line in df.text:
    spls = preprocessor_text(line)
    sentences.append(spls)


In [22]:
len(sentences)

5770

In [31]:
sentences[0]

['американский',
 'аэрокосмический',
 'агентство',
 'огласить',
 'название',
 'космический',
 'миссия',
 'скорый',
 'выбрать',
 'реализация',
 'запуск',
 'состояться',
 'конец',
 'десятилетие',
 'проект',
 'стать',
 'полуфиналист',
 'конкурс',
 'объявить',
 'среди',
 'американский',
 'научный',
 'команда',
 'участвовать',
 'миссия',
 'отобрать',
 'критерий',
 'потенциальный',
 'польза',
 'наука',
 'технический',
 'осуществимость',
 'проект',
 'рамка',
 'программа',
 'заниматься',
 'планирование',
 'миссия',
 'призвать',
 'дать',
 'ответ',
 'фундаментальный',
 'вопрос',
 'происхождение',
 'тело',
 'солнечный',
 'система',
 'возможный',
 'наличие',
 'выбрать',
 'миссия',
 'трансформировать',
 'восприятие',
 'некоторый',
 'активный',
 'сложный',
 'мир',
 'солнечный',
 'система',
 'заявить',
 'томас',
 'зурбутить',
 'помощник',
 'директор',
 'наука',
 'исследование',
 'небесный',
 'тело',
 'помочь',
 'раскрыть',
 'секрет',
 'подобный',
 'объект',
 'образоваться',
 'космос',
 'проект',
 'вы

In [29]:
modelW2V = Word2Vec(sentences=sentences, size=100, window=5, min_count=2)

In [30]:
modelFT = FastText(sentences=sentences, size=100, min_count=2, window=5, workers=8)

In [32]:
w2v_index = annoy.AnnoyIndex(100, 'angular')
ft_index = annoy.AnnoyIndex(100, 'angular')

index_map = {}
counter = 0

for line in range(len(df.text)):
    n_w2v = 0
    n_ft = 0
    spls = df.text[line]
    index_map[counter] = spls
    vector_w2v = np.zeros(100)
    vector_ft = np.zeros(100)
    for word in sentences[line]:
        
        if word in modelW2V.wv:
            vector_w2v += modelW2V.wv[word]
            n_w2v += 1
        if word in modelFT.wv:
            vector_ft += modelFT.wv[word]
            n_ft += 1
            
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        
    
    w2v_index.add_item(counter, vector_w2v)
    ft_index.add_item(counter, vector_ft)
    counter += 1
    
w2v_index.build(10)
ft_index.build(10)    

True

In [47]:
w2v_index

In [33]:
def get_response(question, index, model, index_map):
    question = preprocessor_text(question)
    vector = np.zeros(100)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index.get_nns_by_vector(vector, 5, )
    return [index_map[i] for i in answers]

In [38]:
TEXT = 'цены на газ и нефть'

In [39]:
k = get_response(TEXT, w2v_index, modelW2V, index_map)

In [40]:
k

['Несмотря на эмоциональные высказывания белорусского президента Александра Лукашенко   Россия продолжит обсуждать с Белоруссией вопрос по поводу энергетики  сообщил пресс секретарь российского лидера Дмитрий Песков   Диалог будет продолжаться  тем более  что говорили о том  что продолжатся контакты белорусских коллег непосредственно с компаниями поставщиками нефти     цитирует Пескова ТАСС  Вместе с тем он решил не комментировать заявление Лукашенко о том  что белорусские власти начнут отбирать нефть из транзитной трубы  если Россия в феврале этого года не поставит в нужном объеме топливо в республику   Там иногда  действительно  эмоции берут верх  Но  будучи в Сочи  два президента достигли определенного понимания  вот из этого мы и будем исходить     подчеркнул пресс секретарь Владимира Путина  прокомментирав недавнее высказывание президента Белоруссии  Вместе с тем представитель Кремля напомнил  что стороны также договорились о том  что диалог на уровне вице премьеров стран  которые

In [41]:
TEXT = 'цены на газ и нефть'

In [42]:
get_response(TEXT, ft_index, modelFT, index_map)

['Литва проиграла дело в отношении российской компании  Газпром   Последняя инстанция   Верховный суд Швеции   отклонила финансовые претензии в размере  1 4 млрд  передает ТАСС со ссылкой на министерство энергетики республики  Такую сумму власти страны требовали от  Газпрома  в качестве компенсации за  слишком дорогие поставки  российского газа в период 2004   2012 годов   Верховный суд Швеции в конце декабря принял решение не пересматривать принятое Апелляционным судом Швеции в июле прошлого года решение не аннулировать решение Стокгольмского арбитража в деле Литвы против  Газпрома   Суд не привел мотивы своего решения     говорится в сообщении литовского ведомства  В Вильнюсе уверены  что российская компания пользовалась своим монопольным положением на литовском рынке поставок природного газа и якобы назначала в 2004 2012 годах несправедливую цену за импортное сырье  Но доказать в судебных инстанциях это не удалось  Судебные тяжбы Литвы и  Газпрома  продолжались в течение почти восьм

In [43]:
TEXT = 'принц британии'
get_response(TEXT, ft_index, modelFT, index_map)

['Сын Народной артистки России Елены Яковлевой   27 летний Денис Шальных сообщил в своем инстаграме о нападении группы неизвестных  которое он связал со своим решением публично свести татуировки с лица   Пацаны  меня поймали типы с забитыми  покрытыми татуировками    прим   Газеты      лицами  спросили за лазерное удаление  Не смог отбазариться  Хочется плакать от боли и унижения     написал он  Молодой человек также опубликовал свою фотографию  сделанную после  драки   на которой видно  что у молодого человека заплыл один глаз   Никогда не делайте наколки на лице  Это очень опасно  А комьюнити агрессивное     резюмировал Шальных  По словам людей из окружения Шальных  он начал получать письма с осуждениями от бывших единомышленников и даже угрозы сразу же после первого сеанса лазерной терапии  на котором сводил татуировки с лица  пишет             Сначала ему начали писать угрозы в            а потом такие же татуированные и вовсе подкараулили его недалеко от дома  Обступили  кричали  